In [1]:
import data
import topicmodeling as tp
import gensim as g

Using gpu device 0: GeForce GTX 970 (CNMeM is disabled, cuDNN not available)


In [119]:
l_interested = list(set(data.get_entity_list(data.libinfo.interested_libs)))
print(len(l_interested))
#l_code = list(map(data.get_code, l_interested))
l_doc = list(map(data.get_doc, l_interested))
l_doc = list(map(lambda doc : data.pdoc.extract(doc, stage=data.pdoc.ADVANCED), l_doc))

3979


In [120]:
documents = l_doc
texts = tp.simple_process(documents=documents, stoplist=tp.read_stoplist("../SmartStoplist.txt"))
texts = tp.remove_infrequent(texts, n_times=1)

In [121]:
id2word = g.corpora.Dictionary(texts)
#id2word.save('/tmp/deerwester.dict') # store the id2word, for future reference
print(*list(id2word)[:10])

734 157 1094 316 401 651 51 1056 782 301


In [122]:
corpus = [id2word.doc2bow(text) for text in texts]
#g.corpora.corpusCorpus.serialize('/tmp/deerwester.mm', corpus) # store to disk, for later use
print(*list(corpus)[:10])
tfidf = g.models.tfidfmodel.TfidfModel(corpus)
corpus = [tfidf[bag] for bag in corpus]
print(*list(corpus)[:10])

[(0, 1), (1, 1), (2, 1), (3, 1)] [(4, 1), (5, 1), (6, 1)] [(7, 1), (8, 1), (9, 1)] [(10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)] [(17, 1), (18, 1), (19, 1), (20, 1)] [(21, 1), (22, 1), (23, 1)] [(24, 1), (25, 1), (26, 1), (27, 1), (28, 1)] [(2, 1), (29, 1), (30, 1), (31, 1), (32, 1)] [(2, 1), (3, 1)] [(7, 1), (33, 1), (34, 1), (35, 1)]
[(0, 0.7628244667984156), (1, 0.493458034466283), (2, 0.21387308022274398), (3, 0.35896560647285025)] [(4, 0.6812763365292757), (5, 0.5535053508848635), (6, 0.4790557168295493)] [(7, 0.37854881721187744), (8, 0.7318543350988053), (9, 0.5666480611319289)] [(10, 0.45884116429793026), (11, 0.2767154094081887), (12, 0.43717178069978774), (13, 0.2961800287792112), (14, 0.4109359040475713), (15, 0.3211391957713851), (16, 0.40255793689581393)] [(17, 0.5075897644187175), (18, 0.2754831511310989), (19, 0.4756394287066911), (20, 0.663497398909599)] [(21, 0.6011099968417766), (22, 0.4852135728524555), (23, 0.6350075278424929)] [(24, 0.638733891734

In [ ]:
# load id->word mapping (the id2word), one of the results of step 2 above
#id2word = g.g.corpora.id2word.load_from_text('wiki_en_wordids.txt')
# load corpus iterator
#corpus = g.corpora.MmCorpus('/tmp/deerwester.mm')
#corpus = g.g.corpora.MmCorpus(bz2.BZ2File('wiki_en_tfidf.mm.bz2')) # use this if you compressed the TFIDF output (recommended)

In [123]:
n_topics = 40
## extract 100 LDA topics, using 1 pass and updating once every 1 chunk (10,000 documents)
lda = g.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=n_topics, 
                                 update_every=1, chunksize=10000, passes=5)
## print the most contributing words for n_topic topics
l = list(lda.print_topics(n_topics))
for i, s in l :
    print(i, *s.split(' + '))

0 0.051*seri 0.051*chebyshev 0.042*mask 0.039*multipli 0.038*divid 0.036*fill 0.029*arrai 0.020*polynomi 0.017*repres 0.015*inform
1 0.057*equal 0.051*varianc 0.043*sum 0.037*window 0.032*mask 0.032*arrai 0.027*axi 0.022*neg 0.018*fill_valu 0.018*element
2 0.047*scale 0.025*map 0.025*center 0.025*companion 0.022*count 0.022*numpi 0.018*appli 0.018*featur 0.015*paramet 0.015*partit
3 0.031*statist 0.022*option 0.020*ax 0.019*descript 0.019*ensembl 0.016*user 0.015*flatten 0.015*insert 0.014*subclust 0.014*compon
4 0.043*algorithm 0.036*minim 0.035*order 0.031*log 0.030*probabl 0.029*scalar 0.027*variabl 0.025*function 0.025*quadrat 0.019*section
5 0.041*trim 0.025*dictionari 0.025*sequenc 0.022*digit 0.022*stage 0.020*period 0.020*transform 0.020*error 0.019*stream 0.018*analog
6 0.111*fit 0.088*model 0.059*data 0.045*transform 0.022*estim 0.020*imag 0.018*linear 0.018*filter 0.018*squar 0.017*lowpass
7 0.046*boolean 0.041*filter 0.032*behavior 0.031*dissimilar 0.031*execut 0.030*calcul

In [ ]:
#n_topics = 20
## extract 400 LSI topics; use the default one-pass algorithm
#lsi = g.models.lsimodel.LsiModel(corpus=corpus, id2word=id2word, num_topics=n_topics)
## print the most contributing words (both positively and negatively) for each of the first n_topic topics
#print(*lsi.print_topics(n_topics), sep='\n')

In [124]:
lll = lda
#index = g.similarities.SparseMatrixSimilarity(lll[corpus], num_features=22)
index = g.similarities.Similarity(None, lll[corpus], num_features=n_topics)

In [160]:
i_doc = 3301
show = l_doc
text = texts[i_doc]
sims = index[lll[tfidf[id2word.doc2bow(text)]]]
print(i_doc, '', l_interested[i_doc], sep=' | ')
for i, score in list(sorted(enumerate(sims), key=lambda t : t[1], reverse=True))[:10] :
    print(i, 
          inspect.getmodule(l_interested[i]),
          l_interested[i].__name__,
          "%.3f"%score, 
          show[i].strip(), 
          sep=' | ')

3301 |  | <function linkage at 0x000000001809B6A8>
3301 | <module 'scipy.cluster.hierarchy' from 'C:\\Anaconda3\\envs\\cnn\\lib\\site-packages\\scipy\\cluster\\hierarchy.py'> | linkage | 1.000 | Performs hierarchical/agglomerative clustering on the condensed
distance matrix y.
2618 | <module 'sklearn.decomposition.online_lda' from 'C:\\Anaconda3\\envs\\cnn\\lib\\site-packages\\sklearn\\decomposition\\online_lda.py'> | _update_doc_distribution | 1.000 | E-step: update document-topic distribution.
1359 | <module 'sklearn.cluster.k_means_' from 'C:\\Anaconda3\\envs\\cnn\\lib\\site-packages\\sklearn\\cluster\\k_means_.py'> | _labels_inertia_minibatch | 0.999 | Compute labels and inertia using mini batches.
2677 | <module 'sklearn.neural_network.rbm' from 'C:\\Anaconda3\\envs\\cnn\\lib\\site-packages\\sklearn\\neural_network\\rbm.py'> | gibbs | 0.983 | Perform one Gibbs sampling step.
3 | <module 'scipy.cluster.hierarchy' from 'C:\\Anaconda3\\envs\\cnn\\lib\\site-packages\\scipy\\cluster\\h

In [155]:
print([i_doc for i_doc, doc in enumerate(l_doc) if 'cluster' in doc and 'project' in l_interested[i_doc].__name__])

[1593]


In [137]:
l_interested[1593]

<function sklearn.cluster.bicluster.SpectralBiclustering._project_and_cluster>

In [139]:
tfidf[id2word.doc2bow(texts[1593])]

[(15, 0.4276524485396137),
 (77, 0.5741410809762693),
 (105, 0.2620051940743123),
 (129, 0.5169114161390884),
 (578, 0.3893985985293709)]

In [140]:
lda[corpus[1593]]

[(8, 0.51086741256130108), (37, 0.18945832436536456)]

In [138]:
corpus[1593]

[(15, 0.4276524485396137),
 (77, 0.5741410809762693),
 (105, 0.2620051940743123),
 (129, 0.5169114161390884),
 (578, 0.3893985985293709)]

In [154]:
import inspect
inspect.getmodule(l_interested[i_doc])

<module 'scipy.cluster.hierarchy' from 'C:\\Anaconda3\\envs\\cnn\\lib\\site-packages\\scipy\\cluster\\hierarchy.py'>